In [1]:
from dotenv import load_dotenv
import sqlalchemy
from sqlalchemy import create_engine
import os
import pandas as pd

In [2]:
load_dotenv()

True

In [3]:
redshift_endpoint = "default.695110602608.us-east-2.redshift-serverless.amazonaws.com:5439/dev"

In [4]:

# Connection details
host = 'default.695110602608.us-east-2.redshift-serverless.amazonaws.com'
port = '5439'
database = 'dev'
user = os.environ.get('USERNAME')
password = os.environ.get('PASSWORD')

#Connection details
connection_string = f'redshift+psycopg2://{user}:{password}@{host}:{port}/{database}'
"""
# Establish the connection
try:
    engine = create_engine(connection_string)
    conn = engine.connect()
    print("Connection successful!")
    
    # Perform database operations here
    
except Exception as e:
    print("Error connecting to Redshift:", e)
"""
engine = create_engine(connection_string)
conn = engine.connect()

In [5]:
tables = pd.read_sql("""
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public'
ORDER BY table_name;
""",conn)
tables

,table_name
0,<function table at 0x11485c040>
1,consumer_price_indices_hti
2,exchange_rates_hti
3,haiti_dept_pop_2022
4,hti_pop2019_adm0
5,hti_pop2019_adm1
6,hti_pop2019_adm2
7,hti_pop2019_adm3
8,hti_shp_adm1
9,infrastructure_hti


In [27]:
consumer_price_indices_hti_df = pd.read_csv('data/consumer-price-indices_hti.csv')
consumer_price_indices_hti_df.drop(0,inplace=True)
consumer_price_indices_hti_df.head()

,Iso3,StartDate,EndDate,Area Code,Area Code (M49),Area,Item Code,Item,Months Code,Months,Year Code,Year,Unit,Value,Flag,Note
0,#country+code,#date+start,#date+end,NaN,NaN,#country+name,#indicator+code,#indicator+name,NaN,NaN,NaN,#date+year,#indicator+type,#indicator+value+num,NaN,NaN
1,HTI,2000-01-01,2000-01-31,93.0,'332,Haiti,23013,"Consumer Prices, Food Indices (2015 = 100)",7001.0,January,2000.0,2000,NaN,22.563697,X,base year is 2015
2,HTI,2001-01-01,2001-01-31,93.0,'332,Haiti,23013,"Consumer Prices, Food Indices (2015 = 100)",7001.0,January,2001.0,2001,NaN,26.294924,X,base year is 2015
3,HTI,2002-01-01,2002-01-31,93.0,'332,Haiti,23013,"Consumer Prices, Food Indices (2015 = 100)",7001.0,January,2002.0,2002,NaN,27.770270,X,base year is 2015
4,HTI,2003-01-01,2003-01-31,93.0,'332,Haiti,23013,"Consumer Prices, Food Indices (2015 = 100)",7001.0,January,2003.0,2003,NaN,31.793710,X,base year is 2015


In [40]:
consumer_price_indices_hti_df.to_sql('consumer_price_indices_hti',con=conn,index=False)

807

In [41]:
qc_consumer_price_indices_hti = pd.read_csv('data/qc_consumer-price-indices_hti.csv')
qc_consumer_price_indices_hti.drop(0,inplace=True)
qc_consumer_price_indices_hti.head()

,Iso3,Item Code,Year,Value
1,HTI,23013,2000,22.563697
2,HTI,23013,2001,26.294924
3,HTI,23013,2002,27.770270
4,HTI,23013,2003,31.793710
5,HTI,23013,2004,42.349111


In [42]:
qc_consumer_price_indices_hti.to_sql('qc_consumer_price_indices_hti',con=conn,index=False)

546

In [18]:
os.listdir('data/')

['consumer-price-indices_hti.csv',
 'qc_consumer-price-indices_hti.csv',
 'infrastructure_hti.csv',
 'exchange-rates_hti.csv',
 'deflators_hti.csv',
 'producer-prices_hti.csv']

In [43]:
infrastructure_hti = pd.read_csv('data/infrastructure_hti.csv')
infrastructure_hti.drop(0,inplace=True)
infrastructure_hti.to_sql('infrastructure_hti',con=conn,index=False)
infrastructure_hti.head()

,Country Name,Country ISO3,Year,Indicator Name,Indicator Code,Value
1,Haiti,HTI,2021,"ICT service exports (BoP, current US$)",BX.GSR.CCIS.CD,2000000
2,Haiti,HTI,2020,"ICT service exports (BoP, current US$)",BX.GSR.CCIS.CD,2000000
3,Haiti,HTI,2019,"ICT service exports (BoP, current US$)",BX.GSR.CCIS.CD,21200000
4,Haiti,HTI,2018,"ICT service exports (BoP, current US$)",BX.GSR.CCIS.CD,29400000
5,Haiti,HTI,2017,"ICT service exports (BoP, current US$)",BX.GSR.CCIS.CD,24800000


In [45]:
producer_prices_hti = pd.read_csv('data/producer-prices_hti.csv')
producer_prices_hti.drop(0,inplace=True)
producer_prices_hti.to_sql('producer_prices_hti',con=conn,index=False)
producer_prices_hti.head()

,Iso3,StartDate,EndDate,Area Code,Area Code (M49),Area,Item Code,Item Code (CPC),Item,Element Code,Element,Year Code,Year,Months Code,Months,Unit,Value,Flag
1,HTI,1991-01-01,1991-12-31,93.0,'332,Haiti,56,'0112,Maize (corn),5530.0,Producer Price (LCU/tonne),1991.0,1991,7021.0,Annual value,LCU,2530.000000,A
2,HTI,1992-01-01,1992-12-31,93.0,'332,Haiti,56,'0112,Maize (corn),5530.0,Producer Price (LCU/tonne),1992.0,1992,7021.0,Annual value,LCU,2882.000000,A
3,HTI,1993-01-01,1993-12-31,93.0,'332,Haiti,56,'0112,Maize (corn),5530.0,Producer Price (LCU/tonne),1993.0,1993,7021.0,Annual value,LCU,3278.000000,A
4,HTI,1994-01-01,1994-12-31,93.0,'332,Haiti,56,'0112,Maize (corn),5530.0,Producer Price (LCU/tonne),1994.0,1994,7021.0,Annual value,LCU,4928.000000,A
5,HTI,1995-01-01,1995-12-31,93.0,'332,Haiti,56,'0112,Maize (corn),5530.0,Producer Price (LCU/tonne),1995.0,1995,7021.0,Annual value,LCU,4466.000000,X


In [46]:
producer_prices_hti.dtypes

Iso3                object
StartDate           object
EndDate             object
Area Code          float64
Area Code (M49)     object
Area                object
Item Code           object
Item Code (CPC)     object
Item                object
Element Code       float64
Element             object
Year Code          float64
Year                object
Months Code        float64
Months              object
Unit                object
Value               object
Flag                object
dtype: object

In [5]:
health_sites_haiti = pd.read_csv('data/health_sites_haiti.csv')
print(health_sites_haiti.shape)
health_sites_haiti.head()

(2235, 38)


,X,Y,osm_id,osm_type,completeness,is_in_health_zone,amenity,speciality,addr_street,operator,...,addr_city,name,staff_nurses,changeset_user,wheelchair,beds,url,dispensing,healthcare,operator_type
0,NaN,NaN,540756702,way,15,NaN,hospital,NaN,Rue Soleil 22,NaN,...,Commune Cité Soleil,NaN,NaN,Gregolas555,NaN,NaN,NaN,NaN,hospital,NaN
1,NaN,NaN,268968596,way,21,NaN,clinic,NaN,Rue A (sainte catherine),NaN,...,Cap-Haitien,Docteur Pierre Louis,NaN,Odalma,NaN,NaN,NaN,NaN,NaN,NaN
2,-72.342147,18.539447,945354433,node,18,NaN,dentist,NaN,NaN,NaN,...,NaN,Sourire clinique dentaire,NaN,Xapitoun,NaN,NaN,NaN,NaN,NaN,private_for_profit
3,-72.239821,18.546127,629093436,node,9,NaN,hospital,NaN,NaN,NaN,...,NaN,Centre de santé organisation room,NaN,Manu1400,NaN,NaN,NaN,NaN,NaN,NaN
4,-72.515295,18.771882,1667911659,node,12,NaN,pharmacy,NaN,NaN,NaN,...,NaN,Sagesse Immaculée Pharma,NaN,worldwidewolford,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
health_sites_haiti.to_sql('health_sites_haiti',con=conn,index=False)


PendingRollbackError: Can't reconnect until invalid transaction is rolled back. (Background on this error at: https://sqlalche.me/e/14/8s2b)

In [8]:
haiti_admin1_pop = pd.read_csv("data/hti_admpop_adm1_2022.csv")

In [9]:
haiti_admin1_pop.head()

,adm0code,adm0_fr,adm1code,adm1_fr,Population,Femmes,Hommes,Enfants,Adultes,Filles,Garcons,Femmes adultes,Hommes adultes
0,HTI,Haïti,HT01,Ouest,4210928,2174599,2036329,1853173,2357755,954297,898876,1220302,1137453
1,HTI,Haïti,HT02,Sud-Est,661047,331381,329666,268766,392281,134756,134010,196625,195656
2,HTI,Haïti,HT03,Nord,1115171,567154,548017,533433,581738,271067,262366,296087,285651
3,HTI,Haïti,HT04,Nord-Est,411684,205165,206519,177378,234306,88329,89049,116836,117470
4,HTI,Haïti,HT05,Artibonite,1805210,911855,893355,689304,1115906,347887,341417,563968,551938


In [10]:
haiti_admin1_pop.to_sql("haiti_dept_pop_2022",con=conn,index=False)

10

In [2]:

# Connection details
host = 'wynn.053696186450.us-east-1.redshift-serverless.amazonaws.com'
port = '5439'
database = 'dev'
user = 'player'
password ='tSqOp$*84w3i'

#Connection details
connection_string = f'redshift+psycopg2://{user}:{password}@{host}:{port}/{database}'

engine = create_engine(connection_string)
conn = engine.connect()

In [6]:
df = pd.read_csv("043314093_BostonMedicalCenter_standardcharges.csv")
print(df.dtypes)
print(len(df))
df.head()

CLASS_NM                     object
BILLING_CD                   object
BILLING_CD_TYPE_NM           object
BILLING_DESCRIPTION          object
CPT_MOD                      object
DRG_SOI                     float64
NDC_CD                       object
GROSS_CHG_AMT               float64
PAYER                        object
PLAN                         object
PAYER_NEGOTIATED_CHG_AMT    float64
MIN_NEGOTIATED_CHG_AMT      float64
MAX_NEGOTIATED_CHG_AMT      float64
DISCOUNTED_CASH_PRICE       float64
DW_SRC                        int64
DW_TOP_300                    int64
DW_CMS                        int64
dtype: object
122526


/var/folders/z2/jfjd16v90rx1rm_frwkx4xqh0000gn/T/ipykernel_7008/1960382770.py:1: DtypeWarning: Columns (1,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("043314093_BostonMedicalCenter_standardcharges.csv")


,CLASS_NM,BILLING_CD,BILLING_CD_TYPE_NM,BILLING_DESCRIPTION,CPT_MOD,DRG_SOI,NDC_CD,GROSS_CHG_AMT,PAYER,PLAN,PAYER_NEGOTIATED_CHG_AMT,MIN_NEGOTIATED_CHG_AMT,MAX_NEGOTIATED_CHG_AMT,DISCOUNTED_CASH_PRICE,DW_SRC,DW_TOP_300,DW_CMS
0,INPATIENT,3,MS-DRGV38,ECMO OR TRACHEOSTOMY WITH MV >96 HOURS OR PRIN...,NaN,NaN,NaN,595212.37,BMC HEALTHNET,BMCHN WELL SENSE,106431.74,54123.02,283945.94,148803.0925,1,0,0
1,INPATIENT,3,MS-DRGV38,ECMO OR TRACHEOSTOMY WITH MV >96 HOURS OR PRIN...,NaN,NaN,NaN,595212.37,HEALTH SAFETY NET,HEALTH SAFETY NET BAD DEBT,134684.22,54123.02,283945.94,148803.0925,1,0,0
2,INPATIENT,3,MS-DRGV38,ECMO OR TRACHEOSTOMY WITH MV >96 HOURS OR PRIN...,NaN,NaN,NaN,595212.37,ELDER SERVICE PLAN OTHER,NEIGHBORHOOD PACE (ESP EB),140458.86,54123.02,283945.94,148803.0925,1,0,0
3,INPATIENT,3,MS-DRGV38,ECMO OR TRACHEOSTOMY WITH MV >96 HOURS OR PRIN...,NaN,NaN,NaN,595212.37,BLUE CROSS,BLUE CROSS-MEDICARE ADVANTAGE,142423.47,54123.02,283945.94,148803.0925,1,0,0
4,INPATIENT,3,MS-DRGV38,ECMO OR TRACHEOSTOMY WITH MV >96 HOURS OR PRIN...,NaN,NaN,NaN,595212.37,ELDER SERVICE PLAN OTHER,ELDER SERVICE PLAN OTHER,142423.47,54123.02,283945.94,148803.0925,1,0,0


In [8]:
df.to_sql("BostonMedicalCenter_standardcharges",conn,schema='public')

NotSupportedError: (psycopg2.errors.FeatureNotSupported) Operations on local objects in external schema are not enabled.

[SQL: 
CREATE TABLE public."BostonMedicalCenter_standardcharges" (
	index BIGINT, 
	"CLASS_NM" TEXT, 
	"BILLING_CD" TEXT, 
	"BILLING_CD_TYPE_NM" TEXT, 
	"BILLING_DESCRIPTION" TEXT, 
	"CPT_MOD" TEXT, 
	"DRG_SOI" FLOAT(53), 
	"NDC_CD" TEXT, 
	"GROSS_CHG_AMT" FLOAT(53), 
	"PAYER" TEXT, 
	"PLAN" TEXT, 
	"PAYER_NEGOTIATED_CHG_AMT" FLOAT(53), 
	"MIN_NEGOTIATED_CHG_AMT" FLOAT(53), 
	"MAX_NEGOTIATED_CHG_AMT" FLOAT(53), 
	"DISCOUNTED_CASH_PRICE" FLOAT(53), 
	"DW_SRC" BIGINT, 
	"DW_TOP_300" BIGINT, 
	"DW_CMS" BIGINT
)

]
(Background on this error at: https://sqlalche.me/e/14/tw8g)

In [4]:
pd.read_sql("SELECT * FROM core_rates limit 100;",conn)

,payer_name,negotiation_arrangement,description,additional_information,billing_code,billing_code_type,negotiated_type,negotiated_rate,billing_class,service_code,...,payer_id,state,taxonomy_classification,taxonomy_grouping,taxonomy_specialization,parent_org_name,clean_org_name,city,cbsa_description,network_array
0,Tufts Health Plan,ffs,BLDR IRRIGATION SMPL LVG&/INSTLJ,nan,51700,CPT,fee schedule,147.02,professional,11,...,317,MA,General Acute Care Hospital,Hospitals,None,None,Tufts Medical Center,Boston,Boston-Cambridge-Newton MA-NH,EPO MA
1,Tufts Health Plan,ffs,THYROXINE FR,nan,84439,CPT,fee schedule,16.09,institutional,,...,317,MA,General Acute Care Hospital,Hospitals,None,Baystate Health,Baystate Medical Center,Springfield,Springfield MA,"EPO MA,HMO MA,Limited or Tiered Network EPO MA..."
2,Tufts Health Plan,ffs,IADNA MLT ORGANISMS AMP PRB,nan,87801,CPT,fee schedule,189.8,institutional,,...,317,MA,Clinic/Center,Ambulatory Health Care Facilities,Federally Qualified Health Center (FQHC),None,Lowell Community Health Center,Lowell,Boston-Cambridge-Newton MA-NH,"EPO MA,Navigator by Tufts Health Plan POS MA"
3,Tufts Health Plan,ffs,N-INVAS PHYSIOLOGIC STD UPR/LXTR BLAT MLT LVL BI,nan,93923,CPT,fee schedule,347.75,institutional,,...,317,MA,General Acute Care Hospital,Hospitals,None,Steward Health Care System,Good Samaritan Medical Center,Brockton,Boston-Cambridge-Newton MA-NH,"EPO MA,HMO MA"
4,Tufts Health Plan,ffs,INFLUENZA VACCINE SPLT PRSRV FREE INC ANTIGEN IM,nan,90662,CPT,fee schedule,68.52,institutional,,...,317,MA,General Acute Care Hospital,Hospitals,None,MelroseWakefield Healthcare,Lawrence Memorial Hospital Of Medford,Medford,Boston-Cambridge-Newton MA-NH,"EPO MA,Navigator by Tufts Health Plan POS MA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Tufts Health Plan,ffs,DUP-SCAN AORTA IVC ILIAC VASCL/BPGS COMPL,nan,93978,CPT,fee schedule,595.96,institutional,,...,317,MA,General Acute Care Hospital,Hospitals,None,None,Boston Medical Center,Boston,Boston-Cambridge-Newton MA-NH,"EPO MA,Navigator by Tufts Health Plan POS MA"
96,Tufts Health Plan,ffs,TRANSFERASE ASPARTATE AMINO,nan,84450,CPT,fee schedule,9.24,institutional,,...,317,MA,General Acute Care Hospital,Hospitals,None,Baystate Health,Baystate Franklin Medical Center,Greenfield,Springfield MA,EPO MA
97,Tufts Health Plan,ffs,DUP-SCAN AORTA IVC ILIAC VASCL/BPGS UNI/LMTD,nan,93979,CPT,fee schedule,320.64,institutional,,...,317,MA,Clinic/Center,Ambulatory Health Care Facilities,"Rehabilitation, Cardiac Facilities",None,Steward Medical Group,Brighton,Boston-Cambridge-Newton MA-NH,"EPO MA,Navigator by Tufts Health Plan POS MA"
98,Tufts Health Plan,ffs,BILIRUBIN DIR,nan,82248,CPT,fee schedule,11.72,institutional,,...,317,MA,General Acute Care Hospital,Hospitals,None,Beth Israel Lahey Health,Anna Jaques Hospital,Newburyport,Boston-Cambridge-Newton MA-NH,"EPO MA,HMO MA,Navigator by Tufts Health Plan P..."
